In [2]:
import hashlib
import json
import time

class Block:
    def __init__(self, index, previous_hash, timestamp, transactions, nonce=0):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.transactions = transactions
        self.nonce = nonce
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        block_content = {
            "index": self.index,
            "previous_hash": self.previous_hash,
            "timestamp": self.timestamp,
            "transactions": self.transactions,
            "nonce": self.nonce
        }
        return hashlib.sha256(json.dumps(block_content, sort_keys=True).encode()).hexdigest()

class Blockchain:
    difficulty = 4
    
    def __init__(self):
        self.chain = [self.create_genesis_block()]
        self.pending_transactions = []

    def create_genesis_block(self):
        return Block(0, "0", int(time.time()), [])

    def get_latest_block(self):
        return self.chain[-1]

    def add_transaction(self, sender, recipient, amount):
        transaction = {
            "sender": sender,
            "recipient": recipient,
            "amount": amount
        }
        self.pending_transactions.append(transaction)

    def mine_pending_transactions(self):
        if not self.pending_transactions:
            print("No transactions to mine.")
            return
        
        new_block = Block(len(self.chain), self.get_latest_block().hash, int(time.time()), self.pending_transactions)
        self.mine_block(new_block)
        self.chain.append(new_block)
        self.pending_transactions = []
    
    def mine_block(self, block):
        while block.hash[:self.difficulty] != "0" * self.difficulty:
            block.nonce += 1
            block.hash = block.calculate_hash()

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]
            
            if current_block.hash != current_block.calculate_hash():
                return False
            
            if current_block.previous_hash != previous_block.hash:
                return False
        
        return True

# Test Blockchain
if __name__ == "__main__":
    my_blockchain = Blockchain()
    my_blockchain.add_transaction("Ardit", "Beni", 50)
    my_blockchain.add_transaction("Beni", "Kristi", 30)
    my_blockchain.mine_pending_transactions()
    
    my_blockchain.add_transaction("Kristi", "Donald", 20)
    my_blockchain.mine_pending_transactions()
    
    for block in my_blockchain.chain:
        print(vars(block))
    
    print("Eshte valid blockchain?", my_blockchain.is_chain_valid())


{'index': 0, 'previous_hash': '0', 'timestamp': 1739545349, 'transactions': [], 'nonce': 0, 'hash': '1e0c1894a211f8577c0912b9c358fe5c675cd4e803cf5b36f1628a7bb7679fcc'}
{'index': 1, 'previous_hash': '1e0c1894a211f8577c0912b9c358fe5c675cd4e803cf5b36f1628a7bb7679fcc', 'timestamp': 1739545349, 'transactions': [{'sender': 'Ardit', 'recipient': 'Beni', 'amount': 50}, {'sender': 'Beni', 'recipient': 'Kristi', 'amount': 30}], 'nonce': 14058, 'hash': '00006895727d27da59b12606b92c2734e121f0841ad663f0d7774d58c084a9af'}
{'index': 2, 'previous_hash': '00006895727d27da59b12606b92c2734e121f0841ad663f0d7774d58c084a9af', 'timestamp': 1739545350, 'transactions': [{'sender': 'Kristi', 'recipient': 'Donald', 'amount': 20}], 'nonce': 84146, 'hash': '0000caf3860843fdd39412b080bf0b68961c92544f630f85236873c47050db28'}
Eshte valid blockchain? True
